In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor, Lasso, LinearRegression
from sklearn.ensemble import ExtraTreesRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
''' Reading Data '''
dat = pd.read_csv('../input/ittoday/QuadData.csv')

dat1=dat.copy()
#dat1 = dat.loc[dat['PatternID'] == 1, :]
dat1.set_index('id', inplace=True)

In [ ]:
''' Throw up useless columns '''
dat1 = dat1.drop('DSAlt', axis=1)

In [ ]:
''' plotting the drone movement'''

f, ax = plt.subplots(1,1)
ax.plot(dat1.loc[dat1.ExpID == 5,:].Lat, dat1.loc[dat1.ExpID == 5,:].Lng, label='Position overtime', color='r')
ax.title.set_text('Position Overtime Exp 1')
plt.savefig('Exp1Movement')

In [ ]:
import re 
r = re.compile('Time')
time_col = list(filter(r.match, dat1.columns))
for ti in time_col:
    print(ti, ': ', np.all(np.diff(dat1[ti])>0))


In [ ]:
''' Create Change Col '''
col = dat1.ExpID.unique()
for i in col:
    dat1.loc[dat1['ExpID'] == i, 'DiffLat'] = np.array((0, *np.diff(dat1.loc[dat1['ExpID'] == i, 'Lat'])))
    dat1.loc[dat1['ExpID'] == i, 'DiffLng'] = np.array((0, *np.diff(dat1.loc[dat1['ExpID'] == i, 'Lng'])))
    dat1.loc[dat1['ExpID'] == i, 'DiffAlt'] = np.array((0, *np.diff(dat1.loc[dat1['ExpID'] == i, 'Alt'])))
dat1.groupby('ExpID')['Lat'].apply(lambda x: x.isnull().sum())/dat1.groupby('ExpID')['ExpID'].count()

In [ ]:
usedcol = [1,2, 3, 4,5, 6,7 ]

In [ ]:
dat1['SpeedX'] = dat1['Spd']*np.sin(dat1.Yaw* np.pi/180)
dat1['SpeedY'] = dat1['Spd']*np.cos(dat1.Yaw* np.pi/180)
dat1['SpeedDirXLat'] = dat1['Spd']*((-np.cos(dat1.Yaw*np.pi/180)*np.sin(dat1.Pitch*np.pi/180)*np.sin(dat1.Roll*np.pi/180))-(np.sin(dat1.Yaw*np.pi/180)*np.cos(dat1.Roll*np.pi/180)))
dat1['SpeedDirYLat'] = dat1['Spd']*((-np.sin(dat1.Yaw*np.pi/180)*np.sin(dat1.Pitch*np.pi/180)*np.sin(dat1.Roll*np.pi/180))+(np.cos(dat1.Yaw*np.pi/180)*np.cos(dat1.Roll*np.pi/180)))
dat1['SpeedDirXLng'] = dat1['Spd']*np.cos(dat1.Yaw*np.pi/180)*np.cos(dat1.Pitch*np.pi/180)
dat1['SpeedDirYLng'] = dat1['Spd']*np.sin(dat1.Yaw*np.pi/180)*np.cos(dat1.Pitch*np.pi/180)
dat1['AbsDiffAltX']= abs(dat1['DiffAlt']*np.sin(dat1.Yaw* np.pi/180))
dat1['AbsDiffAltY']= abs(dat1['DiffAlt']*np.cos(dat1.Yaw* np.pi/180))


In [ ]:
dat1.to_csv('train_engineered.csv')

* <a href="./train_engineered.csv"> Download submision File </a>


In [ ]:
''' Modelling ''' 
from sklearn.linear_model import LinearRegression

dat1_train = dat1.loc[(~dat1['Lat'].isnull())&(dat1['ExpID'].isin(usedcol))]
dat1_test = dat1.loc[dat1['Lat'].isnull()]

Xlat_train = dat1_train[['SpeedDirXLat', 'SpeedDirYLat']]
ylat_train = dat1_train['DiffLat']
Xlang_train = dat1_train[['SpeedDirXLng', 'SpeedDirYLng']]
ylang_train = dat1_train['DiffLng']

Xlat_test = dat1_test[['SpeedDirXLat', 'SpeedDirYLat']]
Xlang_test = dat1_test[['SpeedDirXLng', 'SpeedDirYLng']]

modelLat = LinearRegression()
modelLat.fit(Xlat_train, ylat_train)

modelLng = LinearRegression()
modelLng.fit(Xlang_train, ylang_train)


In [ ]:
dat1_test['DiffLat'] = modelLat.predict(Xlat_test)
dat1_test['DiffLng'] = modelLng.predict(Xlang_test)

In [ ]:
for j in dat1_test.ExpID.unique():
    
    XDat = dat1_test.loc[dat1_test['ExpID'] == j, :]
    c = dat1.loc[(dat1_test.loc[dat1_test['ExpID'] == j].index[0]-1)].Lng+dat1_test.loc[dat1_test['ExpID'] == j].DiffLng.iloc[0]
    print(c)
    XDat.iloc[0, 'Lng'] = c
    for i in range(1, len(dat1_test.loc[dat1_test['ExpID'] == j])):
        if (XDat.Alt.iloc[i]<9):
            c = XDat.Lng.iloc[i-1]
        else:
            c = XDat.Lng.iloc[i-1]+XDat.DiffLng.iloc[i]

        XDat.iloc[i, 'Lng'] = c
        print(c)
    
    c = dat1.loc[(dat1_test.loc[dat1_test['ExpID'] == j].index[0]-1)].Lat+dat1_test.loc[dat1_test['ExpID'] == j].DiffLat.iloc[0]
    XDat.iloc[0, 'Lat'] = c
    for  i in range(1, len(dat1_test.loc[dat1_test['ExpID'] == j])):
        if (XDat.Alt.iloc[i]<9):
            c = XDat.Lat.iloc[i-1]
        else:
            c = XDat.Lat.iloc[i-1]+XDat.DiffLat.iloc[i]
        print(c)
        XDat.iloc[i, 'Lat'] = c
        print(i)
    
    print('\n')

    
    dat1_test.loc[dat1_test['ExpID'] == j, :] = XDat
    print(dat1_test.iloc[:, [89, 90]])

In [ ]:
for j in dat1_test.ExpID.unique():
    plt.plot(pd.concat((dat1.loc[(~dat1['Lat'].isnull())&(dat1['ExpID'] == j)], dat1_test.loc[dat1_test['ExpID'] == j]))['Lat'],
             pd.concat((dat1.loc[(~dat1['Lng'].isnull())&(dat1['ExpID'] == j)], dat1_test.loc[dat1_test['ExpID'] == j]))['Lng'])
    plt.plot((dat1.loc[(~dat1['Lat'].isnull())&(dat1['ExpID'] == j)])['Lat'],
             (dat1.loc[(~dat1['Lat'].isnull())&(dat1['ExpID'] == j)])['Lng'])

    plt.show()


In [ ]:
dfduates=dat1_test

In [ ]:
len(dat1_test.ExpID.unique())

In [ ]:
dat1_test[['Alt','DiffAlt','DiffLat','DiffLat','Lat',]].tail(1000)

In [ ]:
dat.Lat.isnull().sum()

In [ ]:
dat.loc[dat['PatternID'] == 1, 'Lat'].isnull().sum()

In [ ]:
dat.loc[dat['PatternID'] == 2, 'Lat'].isnull().sum()

### Prediksi Kotak

In [ ]:
''' Reading Data '''
dat = pd.read_csv('../input/QuadData.csv')
dat2 = dat.loc[dat['PatternID'] == 2, :]
dat2.set_index('id', inplace=True)

In [ ]:
#dat2 = dat.loc[dat['PatternID'] == 2]

In [ ]:
''' Throw up useless columns '''
dat2 = dat2.drop('DSAlt', axis=1)

In [ ]:
import re 
r = re.compile('Time')
time_col = list(filter(r.match, dat2.columns))
for ti in time_col:
    print(ti, ': ', np.all(np.diff(dat2[ti])>0))


In [ ]:
''' Create Change Col '''
col = dat2.ExpID.unique()
for i in col:
    dat2.loc[dat2['ExpID'] == i, 'DiffLat'] = np.array((0, *np.diff(dat2.loc[dat2['ExpID'] == i, 'Lat'])))
    dat2.loc[dat2['ExpID'] == i, 'DiffLng'] = np.array((0, *np.diff(dat2.loc[dat2['ExpID'] == i, 'Lng'])))
    dat2.loc[dat2['ExpID'] == i, 'DiffAlt'] = np.array((0, *np.diff(dat2.loc[dat2['ExpID'] == i, 'Alt'])))
dat2.groupby('ExpID')['Lat'].apply(lambda x: x.isnull().sum())/dat2.groupby('ExpID')['ExpID'].count()

In [ ]:
usedcol = [1,2, 3, 4,5, 6,7 ]

In [ ]:
dat2['SpeedX'] = dat2['Spd']*np.sin(dat2.Yaw* np.pi/180)
dat2['SpeedY'] = dat2['Spd']*np.cos(dat2.Yaw* np.pi/180)
dat2['SpeedDirXLat'] = dat2['Spd']*((-np.cos(dat2.Yaw*np.pi/180)*np.sin(dat2.Pitch*np.pi/180)*np.sin(dat2.Roll*np.pi/180))-(np.sin(dat2.Yaw*np.pi/180)*np.cos(dat2.Roll*np.pi/180)))
dat2['SpeedDirYLat'] = dat2['Spd']*((-np.sin(dat2.Yaw*np.pi/180)*np.sin(dat2.Pitch*np.pi/180)*np.sin(dat2.Roll*np.pi/180))+(np.cos(dat2.Yaw*np.pi/180)*np.cos(dat2.Roll*np.pi/180)))
dat2['SpeedDirXLng'] = dat2['Spd']*np.cos(dat2.Yaw*np.pi/180)*np.cos(dat2.Pitch*np.pi/180)
dat2['SpeedDirYLng'] = dat2['Spd']*np.sin(dat2.Yaw*np.pi/180)*np.cos(dat2.Pitch*np.pi/180)
dat2['AbsDiffAltX']= abs(dat2['DiffAlt']*np.sin(dat2.Yaw* np.pi/180))
dat2['AbsDiffAltY']= abs(dat2['DiffAlt']*np.cos(dat2.Yaw* np.pi/180))


In [ ]:
''' Modelling ''' 
from sklearn.linear_model import LinearRegression

dat2_train = dat2.loc[(~dat2['Lat'].isnull())&(dat2['ExpID'].isin(usedcol))]
dat2_test = dat2.loc[dat2['Lat'].isnull()]

Xlat_train = dat2_train[['SpeedDirXLat', 'SpeedDirYLat', 'AbsDiffAltY']]
ylat_train = dat2_train['DiffLat']
Xlang_train = dat2_train[['SpeedDirXLng', 'SpeedDirYLng', 'AbsDiffAltX']]
ylang_train = dat2_train['DiffLng']



Xlat_test = dat2_test[['SpeedDirXLat', 'SpeedDirYLat', 'AbsDiffAltY' ]]
Xlang_test = dat2_test[['SpeedDirXLat', 'SpeedDirYLat', 'AbsDiffAltX']]



modelLat = LinearRegression()
modelLat.fit(Xlat_train, ylat_train)

modelLng = LinearRegression()
modelLng.fit(Xlang_train, ylang_train)


In [ ]:
dat2_test['DiffLat'] = modelLat.predict(Xlat_test)
dat2_test['DiffLng'] = modelLng.predict(Xlang_test)

In [ ]:
for j in dat2_test.ExpID.unique():
    
    XDat = dat2_test.loc[dat2_test['ExpID'] == j, :]
    c = dat2.loc[(dat2_test.loc[dat2_test['ExpID'] == j].index[0]-1)].Lng+dat2_test.loc[dat2_test['ExpID'] == j].DiffLng.iloc[0]
    print(c)
    XDat.iloc[0, 90] = c
    for i in range(1, len(dat2_test.loc[dat2_test['ExpID'] == j])):
        c = XDat.Lng.iloc[i-1]+XDat.DiffLng.iloc[i]
        XDat.iloc[i, 90] = c
        print(c)
    
    c = dat2.loc[(dat2_test.loc[dat2_test['ExpID'] == j].index[0]-1)].Lat+dat2_test.loc[dat2_test['ExpID'] == j].DiffLat.iloc[0]
    XDat.iloc[0, 89] = c
    for  i in range(1, len(dat2_test.loc[dat2_test['ExpID'] == j])):
        c = XDat.Lat.iloc[i-1]+XDat.DiffLat.iloc[i]
        print(c)
        XDat.iloc[i, 89] = c
        print(i)
    
    print('\n')

    
    dat2_test.loc[dat2_test['ExpID'] == j, :] = XDat
    print(dat2_test.iloc[:, [89, 90]])

In [ ]:
for j in dat2_test.ExpID.unique():
    plt.plot(pd.concat((dat2.loc[(~dat2['Lat'].isnull())&(dat2['ExpID'] == j)], dat2_test.loc[dat2_test['ExpID'] == j]))['Lat'],
             pd.concat((dat2.loc[(~dat2['Lng'].isnull())&(dat2['ExpID'] == j)], dat2_test.loc[dat2_test['ExpID'] == j]))['Lng'])
    plt.plot((dat2.loc[(~dat2['Lat'].isnull())&(dat2['ExpID'] == j)])['Lat'],
             (dat2.loc[(~dat2['Lat'].isnull())&(dat2['ExpID'] == j)])['Lng'])

    plt.show()

In [ ]:
dat2_test.isnull().sum()

In [ ]:
len(pd.concat((dat1_test, dat2_test)))

In [ ]:
dat.Lat.isnull().sum()

In [ ]:
pd.concat((dat1_test, dat2_test))[['Lat', 'Lng']].sort_index().to_csv('mycsvfile.csv',index=True)

* <a href="./mycsvfile.csv"> Download submision File </a>


In [ ]:
#pd.concat((dat1_test, dat2_test))[['Lat', 'Lng']].sort_index()

In [ ]:
#pd.read_csv('submit.csv', index_col=0).sort_index()

In [ ]:
dat1_test.head()